In [113]:
# Save Model Using Pickle
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle

df_fuel = pd.read_csv('fuel_comsumption_2018.csv')
df_flight = pd.read_csv('flights_2018_jan_raw.csv')
df_passengers = pd.read_csv('passengers_nov_dec_jan_raw.csv')

/Users/nickmech/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### **Task 10**: Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?

In [97]:
df_flight.head(5)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-01-01,DL,DL_CODESHARE,DL,3468,9E,N292PQ,3468,15380,TVC,...,207.000,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2018-01-01,DL,DL_CODESHARE,DL,3470,9E,N309PQ,3470,13871,OMA,...,651.000,47.000,0.000,11.000,0.000,0.000,nan,nan,nan,nan
2,2018-01-01,DL,DL_CODESHARE,DL,3471,9E,N337PQ,3471,13487,MSP,...,386.000,23.000,0.000,0.000,0.000,0.000,nan,nan,nan,nan
3,2018-01-01,DL,DL_CODESHARE,DL,3472,9E,N932XJ,3472,12339,IND,...,945.000,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2018-01-01,DL,DL_CODESHARE,DL,3472,9E,N932XJ,3472,14635,RSW,...,945.000,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [98]:
df_flight['mkt_carrier'].unique()

array(['DL', 'B6', 'HA', 'NK', 'AA', 'UA', 'F9', 'WN', 'VX', 'AS', 'G4'],
      dtype=object)

In [99]:
df_flight.shape

(601219, 42)

In [100]:
df_flight.tail()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
601214,2018-01-31,DL,DL_CODESHARE,DL,6307,G7,N367CA,6307,11433,DTW,...,610.000,nan,nan,nan,nan,nan,nan,nan,nan,nan
601215,2018-01-31,DL,DL_CODESHARE,DL,6308,G7,N367CA,6308,13296,MHT,...,610.000,nan,nan,nan,nan,nan,nan,nan,nan,nan
601216,2018-01-31,DL,DL_CODESHARE,DL,6309,G7,N379CA,6309,13487,MSP,...,349.000,nan,nan,nan,nan,nan,nan,nan,nan,nan
601217,2018-01-31,DL,DL_CODESHARE,DL,6310,G7,N374CA,6310,11057,CLT,...,500.000,nan,nan,nan,nan,nan,nan,nan,nan,nan
601218,2018-01-31,DL,DL_CODESHARE,DL,6310,G7,N374CA,6310,11433,DTW,...,500.000,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [101]:
df_flight_mean = df_flight.groupby('mkt_carrier').agg(({'dep_delay': ['mean']}))


In [102]:
df_flight.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [103]:
df_flight_mean

,dep_delay
,mean
mkt_carrier,
AA,8.984
AS,-0.456
B6,20.357
DL,12.193
F9,15.971
G4,10.345
HA,1.720
NK,5.600


In [104]:
df_flight_distance = df_flight.groupby('mkt_carrier').agg(({'distance': ['sum']}))


In [105]:
df_flight_distance

,distance
,sum
mkt_carrier,
AA,111187758.000
AS,24704733.000
B6,24887443.000
DL,89390572.000
F9,10172155.000
G4,6232014.000
HA,4338727.000
NK,14286327.000


In [106]:
df_fuel.head(5)

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,21161.000,09Q,09Q,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",1,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,1504939.000,2703344.000,4208283.000,2018
1,1,21352.000,0WQ,0WQ,Avjet Corporation,1,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,141350.000,4721.000,146071.000,2018
2,1,21652.000,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2018
3,1,21712.000,2HQ,2HQ,Elite Airways LLC,1,89988.000,0.000,0.000,0.000,...,252948.000,0.000,0.000,0.000,0.000,252948.000,1785834.000,0.000,1785834.000,2018
4,1,20408.000,5V,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,1,309045.000,0.000,0.000,0.000,...,837738.000,0.000,0.000,0.000,0.000,837738.000,987918.000,0.000,987918.000,2018


In [107]:
df_fuel['unique_carrier'].unique()

array(['09Q', '0WQ', '27Q', '2HQ', '5V', '5X', '5Y', '8C', '9E', '9S',
       'AA', 'ABX', 'AS', 'B6', 'CP', 'DL', 'EV', 'F9', 'FX', 'G4', 'G7',
       'GFQ', 'GL', 'HA', 'KAQ', 'KD', 'KH', 'KLQ', 'L2', 'M6', 'MQ',
       'N8', 'NC', 'NK', 'OH', 'OO', 'PFQ', 'PO', 'QX', 'SY', 'U7', 'UA',
       'WI', 'WL', 'X9', 'XP', 'YV', 'YX', 'WN', '1BQ', '3EQ'],
      dtype=object)

In [108]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [109]:
df_fuel_gallons = df_fuel.groupby('unique_carrier').agg(({'total_gallons': ['sum']}))
df_fuel_gallons

,total_gallons
,sum
unique_carrier,
09Q,18409760.000
0WQ,91567.000
1BQ,1278377.000
27Q,0.000
2HQ,3758441.000
3EQ,0.000
5V,4637921.000
5X,871068000.000


In [112]:
# df_fuel_gallons2 = df_fuel_gallons.loc[df_fuel_gallons['unique_carrier'].isin(['DL', 'B6', 'HA', 'NK', 'AA', 'UA', 'F9', 'WN', 'VX', 'AS', 'G4'])]

In [125]:
df_y2018 = df_passengers.apply(lambda row: row[df_passengers['year'].isin(['2018'])])

In [126]:
df_y2018m1 = df_y2018.apply(lambda row: row[df_passengers['month'].isin(['1'])])

In [127]:
df_y2018m1

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
28,75.000,75.000,1491450.000,5675.000,4222.000,0.000,0.000,386.000,6393.000,4944.000,...,US,United States,6,673,1,2018,1,1,F,DU
29,75.000,75.000,1491450.000,5680.000,3772.000,0.000,0.000,386.000,6362.000,4583.000,...,US,United States,6,673,1,2018,1,1,F,DU
30,75.000,67.000,1179870.000,4690.000,3528.000,0.000,0.000,610.000,8191.000,6493.000,...,US,United States,6,631,1,2018,1,2,F,DU
31,75.000,70.000,1505140.000,5320.000,3821.000,0.000,0.000,833.000,9453.000,7559.000,...,US,United States,6,638,1,2018,1,2,F,DU
32,75.000,68.000,1374280.000,5291.000,4054.000,1329.000,40.000,469.000,6310.000,4679.000,...,US,United States,6,638,1,2018,1,1,F,DU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601463,75.000,80.000,880000.000,2960.000,842.000,0.000,0.000,325.000,6102.000,4464.000,...,US,United States,6,674,1,2018,1,1,F,DU
601464,75.000,75.000,825000.000,2775.000,795.000,0.000,0.000,325.000,5544.000,4110.000,...,US,United States,6,674,1,2018,1,1,F,DU
601472,75.000,71.000,1548013.000,5396.000,4517.000,0.000,0.000,748.000,10455.000,8308.000,...,US,United States,6,673,1,2018,1,2,F,DU
601473,75.000,73.000,1591619.000,5548.000,2623.000,0.000,0.000,335.000,6826.000,4489.000,...,US,United States,6,673,1,2018,1,1,F,DU


In [128]:
```````````````````

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,75.000,75.000,6962210.000,18313.000,16390.000,483052.000,150581.000,2475.000,27877.000,24967.000,...,US,United States,6,624,1,2018,11,5,F,DU
1,75.000,74.000,2212600.000,8140.000,6919.000,11629.000,34431.000,920.000,11684.000,10076.000,...,US,United States,6,608,1,2018,11,2,F,DU
2,75.000,75.000,3732000.000,14923.000,13551.000,37667.000,3988.000,1310.000,15094.000,13560.000,...,US,United States,6,622,1,2018,11,3,F,DU
3,75.000,75.000,2948400.000,11250.000,9127.000,3294.000,3866.000,361.000,6151.000,4149.000,...,US,United States,6,694,1,2018,11,1,F,DU
4,75.000,75.000,2684300.000,9600.000,7808.000,7126.000,3418.000,224.000,5421.000,3590.000,...,US,United States,6,698,1,2018,11,1,F,DU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601474,75.000,73.000,2634448.000,10950.000,9829.000,5780.000,59326.000,1721.000,20015.000,17763.000,...,US,United States,6,694,1,2018,11,4,F,DU
601475,75.000,75.000,2656827.000,9600.000,8347.000,1386.000,36062.000,1325.000,17004.000,15112.000,...,US,United States,6,698,1,2018,11,3,F,DU
601476,75.000,75.000,3139936.000,12450.000,11192.000,2659.000,4509.000,447.000,6871.000,4951.000,...,US,United States,6,614,1,2018,11,1,F,DU
601477,75.000,75.000,3126175.000,12450.000,11113.000,14934.000,39189.000,679.000,10270.000,7669.000,...,US,United States,6,614,1,2018,11,2,F,DU
